# Chatbot for E-commerce (Logistic Regression)

## 1. Data Analysis

In [10]:
import nltk

import json
import numpy as np
import random

words=[]
classes = []
documents = []

ignore_words = ['?', '!', ".", ",", "'d", "'s", "@" , "#"]

data_file = open('intents.json').read()
intents = json.loads(data_file)

In [11]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi there',
    'How are you',
    'Is anyone there?',
    'Hey',
    'Hola',
    'Hello',
    'Good day'],
   'responses': ['Nice to meet you.',
    'Hello, Good to see you.',
    'Hi there, how can I help you?.'],
   'context': ['']},
  {'tag': 'introduce',
   'patterns': ['What are you?',
    'Who are you?',
    'Can you introduce yourself?',
    "What's your name?"],
   'responses': ['My name is EMO. I am your Online Shopping Assistant.'],
   'context': ['']},
  {'tag': 'goodbye',
   'patterns': ['Bye',
    'See you later',
    'Goodbye',
    'Nice chatting to you, bye',
    'Till next time'],
   'responses': ['See you! Thanks for shopping with us.',
    'Have a nice day',
    'Bye! Come back again soon.'],
   'context': ['']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Awesome, thanks',
    'Thanks for helping me'],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'con

In [12]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [13]:
print("Words: ",words, "\n")
print("Words Size: ", len(words), "\n----------------- \n")

print("Classes: ",classes, "\nClasses Size: ", len(classes),
      "\n----------------- \n")

print("Documents: \nDocument Size: ", len(documents))
for d in documents:
     print(d, "\n")

Words:  ['Hi', 'there', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hey', 'Hola', 'Hello', 'Good', 'day', 'What', 'are', 'you', '?', 'Who', 'are', 'you', '?', 'Can', 'you', 'introduce', 'yourself', '?', 'What', "'s", 'your', 'name', '?', 'Bye', 'See', 'you', 'later', 'Goodbye', 'Nice', 'chatting', 'to', 'you', ',', 'bye', 'Till', 'next', 'time', 'Thanks', 'Thank', 'you', 'That', "'s", 'helpful', 'Awesome', ',', 'thanks', 'Thanks', 'for', 'helping', 'me', 'I', 'want', 'to', 'buy', 'a', 'romantic', 'comedy', 'genre', 'book', 'I', 'want', 'to', 'buy', 'the', 'newest', 'novel', 'you', 'have', 'Do', 'you', 'sell', 'books', '?', 'Can', 'you', 'recommend', 'me', 'some', 'books', '?', 'How', 'can', 'i', 'pay', '?', 'How', 'much', 'money', 'does', 'this', 'cost', '?', 'Check', 'out', 'please', 'I', 'am', 'finished', 'shopping', '.'] 

Words Size:  109 
----------------- 

Classes:  ['greeting', 'introduce', 'goodbye', 'thanks', 'buybooks', 'payment', 'noanswer'] 
Classes Size:  7 
-----

## 2. Preprocess data



In [14]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

print(words)

['a', 'am', 'anyone', 'are', 'awesome', 'book', 'buy', 'bye', 'can', 'chatting', 'check', 'comedy', 'cost', 'day', 'do', 'doe', 'finished', 'for', 'genre', 'good', 'goodbye', 'have', 'hello', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'i', 'introduce', 'is', 'later', 'me', 'money', 'much', 'name', 'newest', 'next', 'nice', 'novel', 'out', 'pay', 'please', 'recommend', 'romantic', 'see', 'sell', 'shopping', 'some', 'thank', 'thanks', 'that', 'the', 'there', 'this', 'till', 'time', 'to', 'want', 'what', 'who', 'you', 'your', 'yourself']


In [6]:
# sort classes
classes = sorted(list(set(classes)))
classes

['buybooks',
 'goodbye',
 'greeting',
 'introduce',
 'noanswer',
 'payment',
 'thanks']

In [7]:
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

65 unique lemmatized words ['a', 'am', 'anyone', 'are', 'awesome', 'book', 'buy', 'bye', 'can', 'chatting', 'check', 'comedy', 'cost', 'day', 'do', 'doe', 'finished', 'for', 'genre', 'good', 'goodbye', 'have', 'hello', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'i', 'introduce', 'is', 'later', 'me', 'money', 'much', 'name', 'newest', 'next', 'nice', 'novel', 'out', 'pay', 'please', 'recommend', 'romantic', 'see', 'sell', 'shopping', 'some', 'thank', 'thanks', 'that', 'the', 'there', 'this', 'till', 'time', 'to', 'want', 'what', 'who', 'you', 'your', 'yourself']


In [8]:
# with open("words_data.json", "w") as words_data:
#     json.dump(words, words_data)
# with open("classes_data.json", "w") as classes_data:
#     json.dump(classes, classes_data)

## 3. Create training and testing data



In [9]:
# create our training data
training = []


In [12]:
for d in documents:
    bow = []
    
    pattern_words = d[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    pattern_words = [w for w in pattern_words if w not in ignore_words]
    #print(pattern_words)
    for w in words:
        bow.append(1) if w in pattern_words else bow.append(0)
    y = classes.index(d[1])
    
    training.append([bow, y])

In [13]:
for t in training:
    print(t)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], 2]
[[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], 2]
[[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], 2]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 2]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 2]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,

In [14]:
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training, dtype=object)

# create train and test lists. X - patterns, Y - intents
X_train = list(training[:,0])
y_train = list(training[:,1])

## 4. Build the Model

In [15]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression

gnb = GaussianNB()
lr = LogisticRegression()

lr.fit(X_train, y_train)


LogisticRegression()

## 5. Predict response

In [16]:
import random
ignore_words = ['?', '!', ".", ",", "'s"]

def clean(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words if word not in ignore_words]
    print("CLEAN: ", sentence_words , "\n")
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, show_details=True):
    # tokenize the pattern
    sentence_words = clean(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    print("BAG: ",bag, "\n")
    return(np.array(bag))

def predict(sentence):
    x_vec = bow(sentence, show_details = False)
    y_pred = lr.predict(np.array([x_vec]))[0]
    intent = classes[y_pred]
    print("PREDICT: ",intent, "\n")

    return intent

In [18]:
def get_response(intent):
    intent_list = intents["intents"]
    for i in intent_list:
        if (i['tag'] == intent):
            result = random.choice(i['responses'])
            break
    return result
def chatbot(word):
    pred = predict(word)
    res = get_response(pred)
    return res

In [ ]:
while(True):
    w = str(input())
    print("You: ", w)
    print("Bot: ",chatbot(w))
    print("-------------")
    